In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [2]:
fdf = pd.read_csv('NY Property data.csv')

In [3]:
df = pd.read_csv('NY Property data.csv')

## Filling Missing Values

In [4]:
## Filling NA Zip Codes
df['ZIP'].fillna(0,inplace=True)
hh = df['ZIP'].tolist()
for i in range(len(hh)):
    if hh[i]==0:
        hh[i]=hh[i-1]
df['ZIP'] = hh

In [5]:
df['ZT'] = df['ZIP'].astype(int).astype(str)+''+df['TAXCLASS'].astype(str)
## Fullval median groupby ZIP and TAXCLASS
fullmedian_zt = df[df['FULLVAL'] != 0].groupby(['ZT'])['FULLVAL'].agg({'count':'count', 'median':'median'})
fullmedian_zt = fullmedian_zt[fullmedian_zt['count']>=10]
## Merge by ZIPTAX
mer = df.merge(fullmedian_zt, how = 'left', left_on='ZT', right_index=True)
## Fill in NA by ZIPTAX median
mer['FULLVAL'].replace(0,np.nan,inplace=True)
mer['FULLVAL'].fillna(mer['median'],inplace=True)
mer['FULLVAL'].fillna(0,inplace=True)

In [6]:
fullmedian_t = df[df['FULLVAL'] != 0].groupby(['TAXCLASS'])['FULLVAL'].agg({'count2':'count', 'median2':'median'})
mer2 = mer.merge(fullmedian_t, how = 'left', left_on='TAXCLASS', right_index=True)
mer2['FULLVAL'].replace(0,np.nan,inplace=True)
mer2['FULLVAL'].fillna(mer2['median2'],inplace=True)

In [7]:
df['FULLVAL']=mer2['FULLVAL']

In [8]:
df['ZT'] = df['ZIP'].astype(int).astype(str)+''+df['TAXCLASS'].astype(str)
## AVTOT median groupby ZIP and TAXCLASS
fullmedian_zt = df[df['AVTOT'] != 0].groupby(['ZT'])['AVTOT'].agg({'count':'count', 'median':'median'})
fullmedian_zt = fullmedian_zt[fullmedian_zt['count']>=10]
## Merge by ZIPTAX
mer = df.merge(fullmedian_zt, how = 'left', left_on='ZT', right_index=True)
## Fill in NA by ZIPTAX median
mer['AVTOT'].replace(0,np.nan,inplace=True)
mer['AVTOT'].fillna(mer['median'],inplace=True)
mer['AVTOT'].fillna(0,inplace=True)
fullmedian_t = df[df['AVTOT'] != 0].groupby(['TAXCLASS'])['AVTOT'].agg({'count2':'count', 'median2':'median'})
mer2 = mer.merge(fullmedian_t, how = 'left', left_on='TAXCLASS', right_index=True)
mer2['AVTOT'].replace(0,np.nan,inplace=True)
mer2['AVTOT'].fillna(mer2['median2'],inplace=True)
df['AVTOT']=mer2['AVTOT']

In [9]:
df['ZT'] = df['ZIP'].astype(int).astype(str)+''+df['TAXCLASS'].astype(str)
## AVLAND median groupby ZIP and TAXCLASS
fullmedian_zt = df[df['AVLAND'] != 0].groupby(['ZT'])['AVLAND'].agg({'count':'count', 'median':'median'})
fullmedian_zt = fullmedian_zt[fullmedian_zt['count']>=10]
## Merge by ZIPTAX
mer = df.merge(fullmedian_zt, how = 'left', left_on='ZT', right_index=True)
## Fill in NA by ZIPTAX median
mer['AVLAND'].replace(0,np.nan,inplace=True)
mer['AVLAND'].fillna(mer['median'],inplace=True)
mer['AVLAND'].fillna(0,inplace=True)
fullmedian_t = df[df['AVLAND'] != 0].groupby(['TAXCLASS'])['AVLAND'].agg({'count2':'count', 'median2':'median'})
mer2 = mer.merge(fullmedian_t, how = 'left', left_on='TAXCLASS', right_index=True)
mer2['AVLAND'].replace(0,np.nan,inplace=True)
mer2['AVLAND'].fillna(mer2['median2'],inplace=True)
df['AVLAND']=mer2['AVLAND']

In [10]:
df['ZT'] = df['ZIP'].astype(int).astype(str)+''+df['TAXCLASS'].astype(str)
## LTFRONT median groupby ZIP and TAXCLASS
fullmedian_zt = df[df['LTFRONT'] != 0].groupby(['ZT'])['LTFRONT'].agg({'count':'count', 'median':'median'})
fullmedian_zt = fullmedian_zt[fullmedian_zt['count']>=10]
## Merge by ZIPTAX
mer = df.merge(fullmedian_zt, how = 'left', left_on='ZT', right_index=True)
## Fill in NA by ZIPTAX median
mer['LTFRONT'].replace(0,np.nan,inplace=True)
mer['LTFRONT'].fillna(mer['median'],inplace=True)
mer['LTFRONT'].fillna(0,inplace=True)
fullmedian_t = df[df['LTFRONT'] != 0].groupby(['TAXCLASS'])['LTFRONT'].agg({'count2':'count', 'median2':'median'})
mer2 = mer.merge(fullmedian_t, how = 'left', left_on='TAXCLASS', right_index=True)
mer2['LTFRONT'].replace(0,np.nan,inplace=True)
mer2['LTFRONT'].fillna(mer2['median2'],inplace=True)
df['LTFRONT']=mer2['LTFRONT']

In [11]:
df['ZT'] = df['ZIP'].astype(int).astype(str)+''+df['TAXCLASS'].astype(str)
## LTDEPTH median groupby ZIP and TAXCLASS
fullmedian_zt = df[df['LTDEPTH'] != 0].groupby(['ZT'])['LTDEPTH'].agg({'count':'count', 'median':'median'})
fullmedian_zt = fullmedian_zt[fullmedian_zt['count']>=10]
## Merge by ZIPTAX
mer = df.merge(fullmedian_zt, how = 'left', left_on='ZT', right_index=True)
## Fill in NA by ZIPTAX median
mer['LTDEPTH'].replace(0,np.nan,inplace=True)
mer['LTDEPTH'].fillna(mer['median'],inplace=True)
mer['LTDEPTH'].fillna(0,inplace=True)
fullmedian_t = df[df['LTDEPTH'] != 0].groupby(['TAXCLASS'])['LTDEPTH'].agg({'count2':'count', 'median2':'median'})
mer2 = mer.merge(fullmedian_t, how = 'left', left_on='TAXCLASS', right_index=True)
mer2['LTDEPTH'].replace(0,np.nan,inplace=True)
mer2['LTDEPTH'].fillna(mer2['median2'],inplace=True)
df['LTDEPTH']=mer2['LTDEPTH']

In [12]:
df['ZT'] = df['ZIP'].astype(int).astype(str)+''+df['TAXCLASS'].astype(str)
## BLDFRONT median groupby ZIP and TAXCLASS
fullmedian_zt = df[df['BLDFRONT'] != 0].groupby(['ZT'])['BLDFRONT'].agg({'count':'count', 'median':'median'})
fullmedian_zt = fullmedian_zt[fullmedian_zt['count']>=10]
## Merge by ZIPTAX
mer = df.merge(fullmedian_zt, how = 'left', left_on='ZT', right_index=True)
## Fill in NA by ZIPTAX median
mer['BLDFRONT'].replace(0,np.nan,inplace=True)
mer['BLDFRONT'].fillna(mer['median'],inplace=True)
mer['BLDFRONT'].fillna(0,inplace=True)
fullmedian_t = df[df['BLDFRONT'] != 0].groupby(['TAXCLASS'])['BLDFRONT'].agg({'count2':'count', 'median2':'median'})
mer2 = mer.merge(fullmedian_t, how = 'left', left_on='TAXCLASS', right_index=True)
mer2['BLDFRONT'].replace(0,np.nan,inplace=True)
mer2['BLDFRONT'].fillna(mer2['median2'],inplace=True)
df['BLDFRONT']=mer2['BLDFRONT']

In [13]:
df['ZT'] = df['ZIP'].astype(int).astype(str)+''+df['TAXCLASS'].astype(str)
## BLDDEPTH median groupby ZIP and TAXCLASS
fullmedian_zt = df[df['BLDDEPTH'] != 0].groupby(['ZT'])['BLDDEPTH'].agg({'count':'count', 'median':'median'})
fullmedian_zt = fullmedian_zt[fullmedian_zt['count']>=10]
## Merge by ZIPTAX
mer = df.merge(fullmedian_zt, how = 'left', left_on='ZT', right_index=True)
## Fill in NA by ZIPTAX median
mer['BLDDEPTH'].replace(0,np.nan,inplace=True)
mer['BLDDEPTH'].fillna(mer['median'],inplace=True)
mer['BLDDEPTH'].fillna(0,inplace=True)
fullmedian_t = df[df['BLDDEPTH'] != 0].groupby(['TAXCLASS'])['BLDDEPTH'].agg({'count2':'count', 'median2':'median'})
mer2 = mer.merge(fullmedian_t, how = 'left', left_on='TAXCLASS', right_index=True)
mer2['BLDDEPTH'].replace(0,np.nan,inplace=True)
mer2['BLDDEPTH'].fillna(mer2['median2'],inplace=True)
df['BLDDEPTH']=mer2['BLDDEPTH']


In [14]:
df['ZT'] = df['ZIP'].astype(int).astype(str)+''+df['TAXCLASS'].astype(str)
## STORIES median groupby ZIP and TAXCLASS
df['STORIES'].replace(np.nan,0,inplace=True)
fullmedian_zt = df[df['STORIES'] != 0].groupby(['ZT'])['STORIES'].agg({'count':'count', 'median':'median'})
fullmedian_zt = fullmedian_zt[fullmedian_zt['count']>=10]
## Merge by ZIPTAX
mer = df.merge(fullmedian_zt, how = 'left', left_on='ZT', right_index=True)
## Fill in NA by ZIPTAX median
mer['STORIES'].replace(0,np.nan,inplace=True)
mer['STORIES'].fillna(mer['median'],inplace=True)
mer['STORIES'].fillna(0,inplace=True)
fullmedian_t = df[df['STORIES'] != 0].groupby(['TAXCLASS'])['STORIES'].agg({'count2':'count', 'median2':'median'})
mer2 = mer.merge(fullmedian_t, how = 'left', left_on='TAXCLASS', right_index=True)
mer2['STORIES'].replace(0,np.nan,inplace=True)
mer2['STORIES'].fillna(mer2['median2'],inplace=True)
df['STORIES']=mer2['STORIES']

## Building variables

In [15]:
final_df = df[['RECORD','FULLVAL','AVLAND', 'AVTOT', 'LTFRONT', 'LTDEPTH', 'BLDFRONT', 'BLDDEPTH', 'ZIP', 'STORIES','B','TAXCLASS']]

In [16]:
final_df['S1'] = final_df['LTFRONT'] *final_df['LTDEPTH']

In [17]:
final_df['S2'] = final_df['BLDFRONT'] *final_df['BLDDEPTH']

In [18]:
final_df['S3'] = final_df['S2'] *final_df['STORIES']

In [19]:
final_df['r1'] = final_df['FULLVAL'] /final_df['S1']
final_df['r2'] = final_df['FULLVAL'] /final_df['S2']
final_df['r3'] = final_df['FULLVAL'] /final_df['S3']
final_df['r4'] = final_df['AVLAND'] /final_df['S1']
final_df['r5'] = final_df['AVLAND'] /final_df['S2']
final_df['r6'] = final_df['AVLAND'] /final_df['S3']
final_df['r7'] = final_df['AVTOT'] /final_df['S1']
final_df['r8'] = final_df['AVTOT'] /final_df['S2']
final_df['r9'] = final_df['AVTOT'] /final_df['S3']

In [20]:
result = []
for i in final_df['ZIP'].astype(int).astype(str):
    result.append(i[:3])
final_df['ZIP3'] = result

In [21]:
g1 = final_df.groupby('ZIP')[['r1','r2','r3','r4','r5','r6','r7','r8','r9']].mean()
g1.columns = ['r1g1','r2g1','r3g1','r4g1','r5g1','r6g1','r7g1','r8g1','r9g1']

In [22]:
g2 = final_df.groupby('ZIP3')[['r1','r2','r3','r4','r5','r6','r7','r8','r9']].mean()
g2.columns = ['r1g2','r2g2','r3g2','r4g2','r5g2','r6g2','r7g2','r8g2','r9g2']

In [23]:
g3 = final_df.groupby('TAXCLASS')[['r1','r2','r3','r4','r5','r6','r7','r8','r9']].mean()
g3.columns = ['r1g3','r2g3','r3g3','r4g3','r5g3','r6g3','r7g3','r8g3','r9g3']

In [24]:
g4 = final_df.groupby('B')[['r1','r2','r3','r4','r5','r6','r7','r8','r9']].mean()
g4.columns = ['r1g4','r2g4','r3g4','r4g4','r5g4','r6g4','r7g4','r8g4','r9g4']

In [25]:
final_df=final_df.merge(g1,how='left',right_index=True,left_on='ZIP')
final_df=final_df.merge(g2,how='left',right_index=True,left_on='ZIP3')
final_df=final_df.merge(g3,how='left',right_index=True,left_on='TAXCLASS')
final_df=final_df.merge(g4,how='left',right_index=True,left_on='B')

In [26]:
final_df['r1g1']=final_df['r1']/final_df['r1g1']
final_df['r2g1']=final_df['r2']/final_df['r2g1']
final_df['r3g1']=final_df['r3']/final_df['r3g1']
final_df['r4g1']=final_df['r4']/final_df['r4g1']
final_df['r5g1']=final_df['r5']/final_df['r5g1']
final_df['r6g1']=final_df['r6']/final_df['r6g1']
final_df['r7g1']=final_df['r7']/final_df['r7g1']
final_df['r8g1']=final_df['r8']/final_df['r8g1']
final_df['r9g1']=final_df['r9']/final_df['r9g1']
final_df['r1g2']=final_df['r1']/final_df['r1g2']
final_df['r2g2']=final_df['r2']/final_df['r2g2']
final_df['r3g2']=final_df['r3']/final_df['r3g2']
final_df['r4g2']=final_df['r4']/final_df['r4g2']
final_df['r5g2']=final_df['r5']/final_df['r5g2']
final_df['r6g2']=final_df['r6']/final_df['r6g2']
final_df['r7g2']=final_df['r7']/final_df['r7g2']
final_df['r8g2']=final_df['r8']/final_df['r8g2']
final_df['r9g2']=final_df['r9']/final_df['r9g2']
final_df['r1g3']=final_df['r1']/final_df['r1g3']
final_df['r2g3']=final_df['r2']/final_df['r2g3']
final_df['r3g3']=final_df['r3']/final_df['r3g3']
final_df['r4g3']=final_df['r4']/final_df['r4g3']
final_df['r5g3']=final_df['r5']/final_df['r5g3']
final_df['r6g3']=final_df['r6']/final_df['r6g3']
final_df['r7g3']=final_df['r7']/final_df['r7g3']
final_df['r8g3']=final_df['r8']/final_df['r8g3']
final_df['r9g3']=final_df['r9']/final_df['r9g3']
final_df['r1g4']=final_df['r1']/final_df['r1g4']
final_df['r2g4']=final_df['r2']/final_df['r2g4']
final_df['r3g4']=final_df['r3']/final_df['r3g4']
final_df['r4g4']=final_df['r4']/final_df['r4g4']
final_df['r5g4']=final_df['r5']/final_df['r5g4']
final_df['r6g4']=final_df['r6']/final_df['r6g4']
final_df['r7g4']=final_df['r7']/final_df['r7g4']
final_df['r8g4']=final_df['r8']/final_df['r8g4']
final_df['r9g4']=final_df['r9']/final_df['r9g4']

## Z Scaling

In [159]:
X = final_df['r1g1']
from sklearn.preprocessing import scale
r1g1_scale = scale(X)
np.mean(r1g1_scale), np.std(r1g1_scale)

(1.5391861644074873e-18, 1.0)

In [160]:
for i in final_df.columns[15:24].tolist()+final_df.columns[25:].tolist():
    final_df[i] = scale(final_df[i])
    #np.mean(final_df[i]), np.std(final_df[i])

## PCA

In [161]:
pdf = final_df[final_df.columns[15:24].tolist()+final_df.columns[25:].tolist()]

In [162]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
pca.fit(pdf)
pca_df = pca.transform(pdf)

In [163]:
pca_df=pd.DataFrame(pca_df)

## Z-scale after PCA

In [164]:
for i in pca_df.columns:
    pca_df[i] = scale(pca_df[i])

## Score 1

In [166]:
pca_df['8']=pca_df.abs().sum(axis=1)

In [167]:
rankpca=pca_df.sort_values(by='8',ascending=True)

In [168]:
rankpca['9']=range(1,1070995)

In [169]:
score = rankpca[['9']]
score.columns=['score1']

## Autoencoder

In [172]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
pca.fit(pdf)
pca_df = pca.transform(pdf)
pca_df=pd.DataFrame(pca_df)
for i in pca_df.columns:
    pca_df[i] = scale(pca_df[i])

In [174]:
import pandas as pd
import numpy as np
from scipy import stats
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler
from pylab import rcParams
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

In [175]:
nb_epoch = 10
batch_size = 128
input_dim = pca_df.shape[1] #num of columns, 8
encoding_dim = 14
hidden_dim = int(encoding_dim / 2) #i.e. 7
learning_rate = 1e-7

In [176]:
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = Dense(hidden_dim, activation="relu")(encoder)
decoder = Dense(hidden_dim, activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

In [177]:
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

cp = ModelCheckpoint(filepath="autoencoder_fraud.h5",
                               save_best_only=True,
                               verbose=0)

tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)

history = autoencoder.fit(pca_df, pca_df,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(pca_df, pca_df),
                    verbose=1,
                    callbacks=[cp, tb]).history

Train on 1070994 samples, validate on 1070994 samples
Epoch 1/10
1070994/1070994 [==============================] - 16s 15us/step - loss: 0.9754 - acc: 0.7772 - val_loss: 0.9674 - val_acc: 0.8194
Epoch 2/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9622 - acc: 0.8512 - val_loss: 0.9518 - val_acc: 0.8741
Epoch 3/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9443 - acc: 0.8761 - val_loss: 0.9353 - val_acc: 0.8885
Epoch 4/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9297 - acc: 0.8816 - val_loss: 0.9230 - val_acc: 0.8865
Epoch 5/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9193 - acc: 0.8784 - val_loss: 0.9146 - val_acc: 0.8167
Epoch 6/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9097 - acc: 0.8655 - val_loss: 0.9052 - val_acc: 0.8752
Epoch 7/10
1070994/1070994 [==============================] - 15s 14us/step - loss: 0.9019 - a

In [178]:
autoencoder = load_model('autoencoder_fraud.h5')

In [179]:
prediction=autoencoder.predict(pca_df)

In [181]:
prediction.shape

(1070994, 8)

In [182]:
diff=prediction-pca_df

In [183]:
diff['8']=diff.abs().sum(axis=1)

In [184]:
rankdiff=diff.sort_values(by='8',ascending=True)

In [185]:
rankdiff['9']=range(1,1070995)

In [186]:
score[2] = rankdiff[['9']]
score.columns=['score1','score2']

In [187]:
score['total']=score['score1']+score['score2']

In [189]:
score.sort_values(by='total',ascending=False).head(20)

,score1,score2,total
632815,1070994,1070994,2141988
565391,1070993,1070993,2141986
1067359,1070992,1070992,2141984
917941,1070991,1070991,2141982
585117,1070990,1070990,2141980
85885,1070989,1070989,2141978
585438,1070988,1070988,2141976
565397,1070987,1070987,2141974
920627,1070986,1070986,2141972
556608,1070985,1070985,2141970


In [149]:
score.to_csv('score.csv')

In [190]:
fdf.iloc[632815]

RECORD                  632816
BBLE                4018420001
B                            4
BLOCK                     1842
LOT                          1
EASEMENT                   NaN
OWNER       864163 REALTY, LLC
BLDGCL                      D9
TAXCLASS                     2
LTFRONT                    157
LTDEPTH                     95
EXT                        NaN
STORIES                      1
FULLVAL               2.93e+06
AVLAND              1.3185e+06
AVTOT               1.3185e+06
EXLAND                       0
EXTOT                        0
EXCD1                      NaN
STADDR          86-55 BROADWAY
ZIP                      11373
EXMPTCL                    NaN
BLDFRONT                     1
BLDDEPTH                     1
AVLAND2             1.2012e+06
AVTOT2              1.2012e+06
EXLAND2                    NaN
EXTOT2                     NaN
EXCD2                      NaN
PERIOD                   FINAL
YEAR                   2010/11
VALTYPE                  AC-TR
ZT      